# Final Project: Honeycomb AKLT as PEPS

Submitted by Yoav Zack, ID 211677398

## `julia` Implementation

We will implement those two wave functions as PEPS in `julia` using `TensorOperations`:

In [ ]:
using LinearAlgebra, TensorOperations, Plots, Printf, BenchmarkTools
theme(:dracula)

First, we define the Honeycomb structure, and some useful functions on it:

In [ ]:
mutable struct HoneycombLattice
    shape::String
    a::Real
    verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}
    bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}}
    ψ::Vector{Array{Float64}}

    function HoneycombLattice(shape::String, a::Real, shift::Tuple, n::Integer, bc::String)
        @assert shape ∈ ("pyramid", "flower", "bricks") "Invalid lattice shape"
        @assert a > 0 "Bond length `a` must be positive"
        @assert length(shift) == 2 "Lattice shift must be of length 2"
        @assert n > 0 "Lattice order `n` must be positive"
        @assert bc ∈ ("periodic", "open") "Invalid lattice boundary condition"

        # define verts location based on lattice shape
        if shape == "pyramid"
            dx = a * sin(π/6)
            dy = a * cos(π/6)
            
            locs = [(0.0, 0.0)]
            for (i,x) in enumerate(range(start=dx, length=n, step=a+dx))
                for y in range(-dy*i, stop=dy*i, length=i+1)
                    append!(locs, [(-x,y), (-x-a,y)])
                end
            end
        
            for (j, y) in enumerate(range(start=-dy*(n-1), stop=dy*(n-1), length=n))
                append!(locs, [(-n*(dx+a)-dx, y)])
            end
        elseif shape == "bricks"
            @assert (n % 2 == 0) || (n == 1) "Number of bricks must be 1 or an even number"
            if n == 1
                locs = [(0.0, 0.0), (0.0, a), (2*a, 0.0), (a, a), (a, 0.0), (2*a, a)]
            else
                locs = [(0.0, 0.0), (0.0, a), (a, 2*a), (a, a)]
                for i in 1:n
                    if i%2==0
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,2*a),(2*a,2*a),(2*a,a)]])
                    else
                        append!(locs, [r.+(i-1).*(a,0) for r in [(a,0),(2*a,0),(2*a,a)]])
                    end
                end
            end            
        end
        
        # add shift to vertex locations and define verts array
        verts = [(ind, loc .+ shift) for (ind, loc) in enumerate(locs)]
        
        # detect bonds
        bonds = detectbonds(verts, a, shape)

        # fix boundary conditions
        @assert bc == "open" "Non-open boundary conditions not implemented yet"
        
        # fill in tensors
        ψ = AKLT_PEPS(verts, bonds)
        
        # create structure
        new(shape, a, verts, bonds, ψ)
    end
end

In [ ]:
function detectbonds(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, a::Real, shape::String)
    bonds = []
    bond_count = ones(Int64, length(verts))
    for vert in verts
        (ind, loc) = vert
        for (ind_j, loc_j) in nearestneighboors(verts[ind:end], vert, a)
            if shape =="bricks" && (ind_j == ind + 1 && ind % 3 == 1 && ind != 1)
                continue
            end
            push!(bonds, ((ind, bond_count[ind]), (ind_j, bond_count[ind_j])))
            bond_count[ind] += 1
            bond_count[ind_j] += 1
        end
    end
    bonds = Vector(Tuple.(bonds))
    return bonds
end

In [ ]:
function nearestneighboors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, a::Real)
    neighbors = []
    (ind, loc) = vert
    for (ind_i, loc_i) in verts
        d = sum((loc .- loc_i).^2)
        if abs(d - a) < 1e-1*a
            push!(neighbors, (ind_i, loc_i))
        end
    end
    return neighbors
end

In [ ]:
function showlattice(fig::Plots.Plot, lat::HoneycombLattice, dolabel::Bool)
    x = [loc[1] for (ind, loc) in lat.verts]
    y = [loc[2] for (ind, loc) in lat.verts]
    for bond in lat.bonds
        inds = [b[1] for b in bond]
        dx = [lat.verts[i][2][1] for i in inds]
        dy = [lat.verts[i][2][2] for i in inds]
        plot!(fig, dx, dy, label="", color="cyan")
    end
    scatter!(fig, x, y, aspect_ratio=1, label="", color="dark red")

    if dolabel
        for (ind, loc) in lat.verts
            annotate!(fig, loc[1], loc[2], ind)
        end
    end
    return fig
end

The tensors are defined using the following functions:

In [ ]:
function AKLT_PEPS(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}})
    """
    Builds a PEPS tensor array of the AKLT model for a given set of vertices and the bonds between them, assuming a HoneycombLattice structure (i.e. three connections at most)
    """
    
     # adds a Θ tensor per site. Θ tensors have 4 indices: σ,a,b,c. The only real one is σ, the rest are virtual
    ψ = [projectionOp() for i in 1:length(verts)]

    # multiply the tensors by the bond matrices: antisymmetric spin-1/2
    Σ = zeros(2,2); Σ[1,2] = 1/sqrt(2); Σ[2,1] = -1/sqrt(2);
    for bond in bonds
        (ind, conn) = bond[1]
        A = ψ[ind]
        if conn == 1
            @tensor B[σ,ap,b,c] := A[σ,a,b,c] * Σ[a,ap]
        elseif conn == 2
            @tensor B[σ,a,bp,c] := A[σ,a,b,c] * Σ[b,bp]
        elseif conn == 3
            @tensor B[σ,a,b,cp] := A[σ,a,b,c] * Σ[c,cp]
        end
        ψ[ind] = B
    end

    # finally, set the boundary legs of each vertex to specific value, if needed
    σ_sign = 2
    for (ind, A) in enumerate(ψ)
        c, _ = countneighbors(verts, verts[ind], bonds)
        if c == 0
            A = reshape(A[:,σ_sign,σ_sign,σ_sign], 4,1,1,1)
        elseif c == 1
            A = reshape(A[:,:,σ_sign,σ_sign], 4,2,1,1)
        elseif c == 2
            A = reshape(A[:,:,:,σ_sign], 4,2,2,1)
        end
        ψ[ind] = A
    end
    return ψ
end

In [ ]:
function projectionOp()
    Θ = zeros(4,2,2,2)
    for a in 1:2
        for b in 1:2
            for c in 1:2
                Θ[1,a,b,c] = (a+b+c == 3)
                Θ[2,a,b,c] = (a+b+c == 4)/sqrt(3)
                Θ[3,a,b,c] = (a+b+c == 5)/sqrt(3)
                Θ[4,a,b,c] = (a+b+c == 6)
            end
        end
    end
    return Θ
end

In [ ]:
function countneighbors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}})
    c = 0
    vert_bonds = []
    (ind, loc) = vert
    for bond in bonds
        inds = [b[1] for b in bond]
        if ind ∈ inds
            c += 1
            push!(vert_bonds, bond)
        end
    end
    return c, vert_bonds
end

For example, here are two possible lattices:

In [ ]:
lat = HoneycombLattice("bricks", 1, (5.0,1.0), 1, "open")
fig = showlattice(scatter(), lat, true)

In [ ]:
lat = HoneycombLattice("pyramid", 1, (5.0,1.0), 2, "open")
fig = scatter()
fig = showlattice(fig, lat, true)

In order to verity that this is indeed only a factor, we will add a single site with only one connection, annd verify that it indeed has $〈S_z〉=\pm 2\cdot \frac 23$, i.e. twice what is seen above:

In [ ]:
Sz = diagm([3,1,-1,-3]./2)
I = diagm([1,1,1,1])
lat.verts = [(1,(0.0,0.0)),(2,(1.0,0.0))]
lat.bonds = [((1,1),(2,1))]
lat.ψ = AKLT_PEPS(lat.verts, lat.bonds)
@tensor g[σ1, σ2] := lat.ψ[1][σ1,a,1,1] * lat.ψ[2][σ2,a,1,1]
display(g)
@tensor sz = g[σ1,σ2] * Sz[σ1,a] * I[σ2,b] * g[a,b]
@tensor n = g[σ1,σ2] * I[σ1,a] * I[σ2,b] * g[a,b]
display(sz/n)
fig, Olist = mapexpect(lat, Sz, 2)
display(fig)